In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import cv2
import scipy.signal
import scipy.ndimage
import os
# helpers.py is one level up in the directory structure so we need to tell Python were to find it
import sys
sys.path.append("../../")
import helpers

# Convolution
In addition to filters that only consider a single input pixel at a time (such as in the brightness and saturation filters), we can also design filters that take into account a neighbourhood around a pixel. A popular way of formulating and applying these filters is through the use of the convolution operator. The convolution operator applies a weighted sum to the surrounding pixels for each output pixel. The weights of the weighted sum are organized in an array (called kernel). The kernel is centered around the pixel that is currently computed (hence the size of the kernel should always be uneven) - compare the slides on box filtering.

## Exercise 6 (2 points)
Implement a function that applies a convolution to a 1D array using a given kernel. The output array should be of the same size as the input and values outside the input range should be interpreted as zero.

In [87]:
def apply_convolution1D(data, kernel):
    near = len(kernel) // 2
    convolution = np.zeros(data.shape)

    for x in range(data.shape[0]):
        if x < near:
            first_weight = near - x
            first_nbor = 0
        else:
            first_weight = 0
            first_nbor = x - near
        
        nbors = data[first_nbor:x+1 + near]
        weights = kernel[first_weight:first_weight+len(nbors)]
        convolution[x] = np.sum(nbors * weights)
        
    
    return convolution


data = np.array([0.7, 0.3, 0.4, 0.6, 0.1])
kernel = np.array([-1, 2, -1]) / 4
print(f"Input:              {data}")
print(f"Your solution:      {apply_convolution1D(data, kernel)}")

Input:              [0.7 0.3 0.4 0.6 0.1]
Your solution:      [ 0.275 -0.125 -0.025  0.175 -0.1  ]


### Testing your solution of exercise 6
Very by hand (pen and paper) that your solution is correct. You are free to try different inputs and kernels as well.

In [85]:
np.random.seed(42)
data = np.random.rand(10)
kernel = np.array([-1, 2, -1]) / 4
result = apply_convolution1D(data, kernel)
assert(type(result) == np.ndarray) # np.array
assert(len(result) == 10)


## Exercise 7 (2 points)
Convolutions can also be applied to 2 dimensional data using a 2D kernel. Implement a function that applies the convolution operator in 2D.

In [94]:
def apply_convolution2D(data, kernel):
    near = tuple(dim//2 for dim in kernel.shape)
    convolution = np.zeros(data.shape)
    
    for y in range(data.shape[0]):
        if y < near[0]:
            first_yweight = near[0] - y
            first_ynbor = 0
        else:
            first_yweight = 0
            first_ynbor = y - near[0]

        for x in range(data.shape[1]):
            if x < near[1]:
                first_xweight = near[1] - x
                first_xnbor = 0
            else:
                first_xweight = 0
                first_xnbor = x - near[1]

            nbors = data[first_ynbor:y+1 + near[0], first_xnbor:x+1 + near[1]]
            weights = kernel[first_yweight:first_yweight+nbors.shape[0], first_xweight:first_xweight+nbors.shape[1]]
            convolution[y, x] = np.sum(nbors * weights)
    
    return convolution

np.random.seed(42)
data = np.array([
    [0.4, 0.9, 0.7, 0.6, 0.2],
    [0.2, 0.5, 0.9, 0.6, 0.7],
    [0.0, 1.0, 0.8, 0.2, 0.1],
    [0.1, 0.3, 0.5, 0.4, 0.3],
    [0.6, 0.1, 0.3, 0.3, 0.5]
])
kernel = np.array([
    [0, 1, 0],
    [1, 2, 1],
    [0, 1, 0],
])

result = apply_convolution2D(data, kernel)
print(f"Input:\n{data}")
print(f"\nYour solution:\n{result}")

Input:
[[0.4 0.9 0.7 0.6 0.2]
 [0.2 0.5 0.9 0.6 0.7]
 [0.  1.  0.8 0.2 0.1]
 [0.1 0.3 0.5 0.4 0.3]
 [0.6 0.1 0.3 0.3 0.5]]

Your solution:
[[1.9 3.4 3.8 2.7 1.7]
 [1.3 4.  4.4 3.6 2.3]
 [1.3 3.6 4.2 2.3 1.4]
 [1.1 2.3 2.8 2.1 1.6]
 [1.4 1.4 1.5 1.8 1.6]]


### Testing your solution of exercise 7
Verify by hand (pen and paper) that your solution is correct. You are free to try different inputs and kernels as well.

In [95]:
# Basic check to verify that the returned type is a numpy array of the correct size.
data = np.random.rand(5, 5)
kernel = np.random.rand(3, 3)
result = apply_convolution2D(data, kernel)
assert(type(result) == np.ndarray) # np.array
assert(result.shape == (5, 5))


## Exercise 8: Box blur (1 point)
Convolutions can be used to implement various image filters such as blurs. In the lecture, we discussed a box blur, where the output value is the mean of the surrounding pixels (including the center/input pixel). Implement a function that returns a box blur kernel of size $2N+1$ ($N$ to the left and $N$ to the right).

In [ ]:
def box_blur_kernel1D(N):
    # YOUR CODE HERE
    raise NotImplementedError()

np.random.seed(42)
x = np.linspace(-0.1, 1.1) # Slightly outside the visible range so we don't see the boundary effects
y = 0.8 * np.sin(10 * x) + 0.2 * (2 * np.random.rand(x.shape[0]) - 1)

plt.figure(figsize=helpers.default_fig_size)
plt.grid(True)
plt.plot(x, y, '-', linewidth=2)
plt.plot(x, np.convolve(y, box_blur_kernel1D(3), mode="same"), '--', linewidth=2)
plt.xlim(0, 1)
plt.ylim(-1, 1)
plt.legend(["Input data", "Box blurred (your solution)"])
plt.title("Box blur applied to noisy input data (sin curve)")
plt.show()

### Testing your solution of exercise 8
If you implemented exercise 8 correctly then the orange line should be a smoother version of the blue line.

In [ ]:
# Basic check to verify that the returned type is a numpy array of the correct size.
kernel = box_blur_kernel1D(2)
assert(type(kernel) == np.ndarray) # np.array
assert(kernel.shape == (5,))


## Exercise 9 (1 point)
Implement a function that returns a kernel for a **2D** box blur.

In [ ]:
def box_blur_kernel2D(N):
    # Return kernel of size (2N+1)x(2N+1)
    # YOUR CODE HERE
    raise NotImplementedError()

new_york_image = helpers.imread_normalized_float_grayscale(os.path.join(helpers.dataset_folder, "week1", "colors", "newyork.jpg"), 0.25) # Downscale by 4 in each direction = 16 times less pixels.

new_york_blurred5 = scipy.signal.convolve2d(new_york_image, box_blur_kernel2D(2))
%time new_york_blurred19 = scipy.signal.convolve2d(new_york_image, box_blur_kernel2D(9))

helpers.show_images({ "Input": new_york_image , "Slightly blurred (your solution)": new_york_blurred5, "Heavily blurred (your solution)":  new_york_blurred19 }, nrows=1, ncols=3)

# Release some memory
del new_york_blurred5
del new_york_blurred19

### Testing your solution of exercise 9
If you implemented exercise 9 correctly then the images should show a slightly blurred and a more severly blurred version of the input image. The edges of the image will appear dark because the convolution assumes that regions outside the image are black.

In [ ]:
# Basic check to verify that the returned type is a numpy array of the correct size.
kernel = box_blur_kernel2D(2)
assert(type(kernel) == np.ndarray) # np.array
assert(kernel.shape == (5, 5))


# Exercise 10 (3 points)
If you zoom in on the blurred image you will notice that the box filter creates blocky artifacts. Another popular blur filter, the Gaussian blur, prevents artifacts and results in a smoother image. Implement a function that returns a 2D Gaussian kernel as was described in the lecture. 

**Note:** Make sure that the resulting kernel is normalized (weights sum up to 1).

In [ ]:
def gaussian_blur_kernel2D(standard_deviation, N):
    # Return kernel of size (2N+1 x 2N+1)
    # YOUR CODE HERE
    raise NotImplementedError()

std = 3
new_york_blurred = scipy.signal.convolve2d(new_york_image, gaussian_blur_kernel2D(std, 1+8*std), mode="same")
helpers.show_images({ "Input": new_york_image, "Your solution": new_york_blurred }, nrows=1, ncols=2)

# Release some memory
del new_york_blurred

# Testing your solution of exercise 10
If you implemented exercise 10 correctly then your solution should look blurry but without the horizontal/vertical line artefacts. We provide a couple of basic tests to ensure that some of the basic requirements are met.

In [ ]:
# Basic check to verify that the returned type is a numpy array of the correct size.
std = 4
N = 4*std
kernel = gaussian_blur_kernel2D(std, N)
assert(type(kernel) == np.ndarray) # np.array
assert(kernel.shape == (2*N+1, 2*N+1))
assert(np.abs(np.sum(kernel) - 1) < 0.00001) # The kernel should sum up to 1


## Efficient 2D blurs
The blur filter that you just implemented will take some time to execute, even on a low resolution image. The amount of operations that the `convolve2D` function has to execute depends on the size of the kernel. However because the kernel is square, each time we double $N$ the size of the kernel quadruples. This problem only gets worse on higher resolution images because a larger kernel is required to get the same visual effect.

Let's take a look at what happens when we apply your Gaussian blur to a higher resolution image.

In [ ]:
new_york_image_halfres = helpers.imread_normalized_float_grayscale(os.path.join(helpers.dataset_folder, "week1", "colors", "newyork.jpg"), 0.5) # Downscale by 2 in each direction = 4 times less pixels.

std = 5
kernel = gaussian_blur_kernel2D(std, 4*std)
print("convolve2d")
%time im1 = scipy.signal.convolve2d(new_york_image_halfres, kernel, mode="same")

print("\ngaussian_filter")
%time im2 = scipy.ndimage.gaussian_filter(new_york_image_halfres, std, mode="constant", cval=0, truncate=4)

helpers.show_images({"Convolution (your solution of exercise 10)": im1, "scipy.ndimage.gaussian_filter": im2},nrows=1,ncols=2)

Your Gaussian blur may take multiple seconds to execute compared to just a fraction of a second for the `ndimage` implementation. But numpy is really fast right, so how is `scipy.ndimage.gaussian_filter` doing this?

The trick to making a 2D filter fast is its *separability*. A kernel is separable if we can write it as a convolution of multiple lower resolution (1 dimensional) kernels. The result of convolving with the those smaller kernels in succession is the same as convolving with the original kernel. As the convolution operator for each of the smaller kernels is much cheaper, we obtain a significant speedup. More formally:

$$
A \circledast (M_1 \circledast M_2) = (A \circledast M_1) \circledast M_1 \text{ , where }\circledast\text{ is the convolution operator}
$$

Of course, this only works if $M$ can be written as the multiplication of two (smaller) kernels. Such as:
$$
\left(\begin{matrix}3&6&9\\4&8&12\\5&10&15\end{matrix}\right) =
\left(\begin{matrix}3\\4\\5\end{matrix}\right) \circledast
\left(\begin{matrix}1&2&3\end{matrix}\right)
$$

This is the case for both the box blur and the Gaussian blur kernels. Instead of performing a single convolution with a $NxN$ kernel we can *separate* them into a horizontal ($Nx1$) and a vertical blur ($1xN$) kernel. The 2D box blur for example can be written as:

$$
\left(\begin{matrix}\frac{1}{9}&\frac{1}{9}&\frac{1}{9}\\\frac{1}{9}&\frac{1}{9}&\frac{1}{9}\\\frac{1}{9}&\frac{1}{9}&\frac{1}{9}\end{matrix}\right) =
\left(\begin{matrix}\frac{1}{3}\\\frac{1}{3}\\\frac{1}{3}\end{matrix}\right) \circledast
\left(\begin{matrix}\frac{1}{3}&\frac{1}{3}&\frac{1}{3}\end{matrix}\right)
$$

## Exercise 11 (2 points)
Implement a function that applies a 2D **box** blur using two separate kernels (a vertical and a horizontal kernel). The results should match that of the earlier assignments. For this exercise you *are* allowed to use `scipy.signal.convolve2d`.

**Note:** When you pass a 1 dimensional kernel to `scipy.signal.convolve2D` it will not know along which axis to apply the kernel. So instead always create a 2 dimensional kernel (e.g. `1 x (2N+1)` and `(2N+1) x 1)`).

In [ ]:
def separable_box_blur2D(gray_image, N):
    # Blur by (2N+1 x 2N+1) kernel
    # YOUR CODE HERE
    raise NotImplementedError()

N = 10
print("Time taken with 2D kernel (your solution from exercise 9)")
%time blurred_image = scipy.signal.convolve2d(new_york_image_halfres, box_blur_kernel2D(N), mode="same")

print("\nTime taken with separable kernel (your solution):")
%time blurred_image_separable = separable_box_blur2D(new_york_image_halfres, N)

print(f"\nDifference compared to your earlier solution: {helpers.SSD_per_pixel(blurred_image, blurred_image_separable)}")
helpers.show_images({ "Separable convolution box blur (your solution)": blurred_image_separable, "2D convolution box blur (from exercise 9)": blurred_image }, nrows=1, ncols=2)

### Testing your solution of exercise 11
If you implemented exercise 11 correctly then your solution should match your result from exercise 9. However it should now take a lot less time to compute the same blurred image.

In [ ]:
# Basic checks to test if the output has the correct dimensions
random_image = np.random.rand(100, 100)
blurred_random_image = separable_box_blur2D(random_image, 2)
assert(type(blurred_random_image) == np.ndarray) # np.array
assert(blurred_random_image.shape == (100, 100))


## Exercise 12 (3 points)
Implement a 2D **Gaussian** blur as a combination of two 1D Gaussian blurs. You can find the formula for the 1D Gaussian blur in the slides. For this exercise you *are* allowed to use `scipy.signal.convolve2d`.

In [ ]:
def apply_gaussian_blur2D(gray_image, standard_deviation, N):
    # YOUR CODE HERE
    raise NotImplementedError()

std = 5
N = 4*std
print("Time taken with 2D kernel (your solution from exercise 10)")
%time blurred_image = scipy.signal.convolve2d(new_york_image_halfres, gaussian_blur_kernel2D(std, N), mode="same")

print("\nTime taken with separable kernel (your solution):")
%time blurred_image_separable = apply_gaussian_blur2D(new_york_image_halfres, std, N)

print(f"\nDifference compared to your earlier solution: {helpers.SSD_per_pixel(blurred_image, blurred_image_separable)}")

helpers.show_images({ "Separable convolution (your solution)": blurred_image_separable, "2D convolution Gaussian blur (from exercise 10)": blurred_image }, nrows=1, ncols=2)

# Clean up some memory
del blurred_image
del blurred_image_separable

### Testing your solution of exercise 12
If you implemented exercise 12 correctly then your solution should match your result from exercise 10. However it should now take a less time to compute the same blurred image. Your code should meet the following performance requirements to receive points for this exercise.

In [ ]:
# Basic checks to test if the output has the correct dimensions
random_image = np.random.rand(100, 100)
blurred_random_image = apply_gaussian_blur2D(random_image, 2, 8)
assert(type(blurred_random_image) == np.ndarray) # np.array
assert(blurred_random_image.shape == (100, 100))

timer1 = helpers.Timer()
N = 16
with timer1:
    apply_gaussian_blur2D(new_york_image_halfres, std, N)
timer2 = helpers.Timer()
with timer2:
    scipy.signal.convolve2d(new_york_image_halfres, np.zeros((2*N+1, 2*N+1)), mode="same")
print(f"Your solution was {timer2.elapsed() / timer1.elapsed()} times faster than 2D convolution (must be at least {N / 4} faster)")

N = 20
timer1 = helpers.Timer()
with timer1:
    apply_gaussian_blur2D(new_york_image_halfres, std, N)
timer2 = helpers.Timer()
with timer2:
    scipy.signal.convolve2d(new_york_image_halfres, np.zeros((2*N+1, 2*N+1)), mode="same")
print(f"Your solution was {timer2.elapsed() / timer1.elapsed()} times faster than 2D convolution (must be at least {N / 4} faster)")


# Numerical derivatives
In high-school you have learned how to analytically compute the derivative of a function $f(x)$. For images we do not know the exact function but we can view the pixels as measurements of an unknown function. We can approximate the derivative of the underlying function by computing the derivatives numerically. The simplest method of numerically approximate derivatives is to use finite differences. 

## Exercise 13 (1 point)
Two common estimates for computing the derivatives are the Newton difference quotient and the symmetric difference quotient:
$$f'(x)=\frac{f(x + \Delta x) - f(x)}{\Delta x} \text{  and  } f'(x)=\frac{f(x + \Delta x) - f(x - \Delta x)}{2h} \text{ respectively}$$

We can implement these two derivative filters as convolution kernels. Implement the two given functions such that they return a convolution **kernel** (`np.array`) that computes the newton or symmetric numerical derivatves respectively.

**Note:** We use `np.correlate` instead of `np.convolve`. The difference between these two operations is that `np.convolve` applies the kernel in reverse order. This difference is only relevant to non-symmetric kernels (such as these) where the correlation operator is more intuitive.

In [ ]:
def finite_difference_newton_kernel():
    # return np.array(...)
    # YOUR CODE HERE
    raise NotImplementedError()

def finite_difference_symmetric_kernel():
    # return np.array(...)
    # YOUR CODE HERE
    raise NotImplementedError()

x = np.linspace(0, 1, num=50) # Slightly outside the visible range so we don't see the boundary effects
y = 2.5 * x**2 - 1.2 * x  + np.sin(x * 10)

plt.figure(figsize=helpers.default_fig_size)
plt.grid(True)
plt.plot(x*50, y, '-', linewidth=2)
plt.plot(x*50, np.correlate(y, finite_difference_newton_kernel(), mode="same"), '--', linewidth=2)
plt.plot(x*50, np.correlate(y, finite_difference_symmetric_kernel(), mode="same"), '--', linewidth=2)
plt.legend(["Input data", "Newton (your solution)", "Symmetric (your solution)"])
plt.title("2-point finite difference estimates")
plt.show()

### Testing your solution of exercise 13
If you implemented exercise 13 correctly then the green and yellow lines should approximate the derivative of the blue line. The functions may look weird at the endpoints ($x=0$ and $x=50$) because it is missing neighbour data.

# Image gradients
Similar to the 1D case we can also use numerical differences to compute partial derivatives of a multi dimensional function. The vector containing the partial derivatives is called the gradient vector. In the 2D case (such as images) the gradient is defined as $\nabla f = \begin{pmatrix}g_x \\ g_y\end{pmatrix}$.

The previous estimates for 1D signals can be extended to 2D. A commonly used filter to compute image derivatives is the Sobel operator:

$$
\begin{pmatrix}
-1 & 0 & +1 \\
-2 & 0 & +2 \\
-1 & 0 & +1 \\
\end{pmatrix}
\text{ and }
\begin{pmatrix}
-1 & -2 & -1 \\
0 & 0 & 0 \\
+1 & +2 & +1 \\
\end{pmatrix}
$$

## Exercise 14 (2 points)
The Sobel operator is an edge detection filter: it highlights edges in the image. The operator is defined as the magnitude (length) of the gradient vector. Implement the Sobel operator. For this exercise you *are* allowed to use `scipy.signal.correlate2d`.

In [ ]:
def sobel_operator(gray_image):
    # YOUR CODE HERE
    raise NotImplementedError()

new_york_sobel = sobel_operator(new_york_image_halfres)

helpers.show_images({
    "Input": new_york_image_halfres,
    "Your solution": helpers.normalize_image(new_york_sobel)
}, nrows=1, ncols=2)

### Testing your solution of exercise 14
If you implemented exercise 14 then your result should be a grayscale image where lines are contours (lines/edges) are highlighted in light and the smooth areas (such as the sky) are dark.